<a href="https://colab.research.google.com/github/FaisalT0435/Sistem-Deteksi-Hama-menggunakan-Deep-learning-dan-IoT/blob/main/Mask%20RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
!pip install pycocotools
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
import torch
import os

In [93]:
os.chdir('/content/drive/MyDrive/Ta/Mask_RCNN/Dataset')

In [94]:
!pwd

/content/drive/MyDrive/Ta/Mask_RCNN/Dataset


In [95]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 5e-4
BATCH_SIZE = 8
NUM_EPOCHS = 1
NUM_WORKERS = 4
CHECKPOINT_FILE = "1152KaggleBest.pth.tar"
PIN_MEMORY = True
SAVE_MODEL = False
LOAD_MODEL = False
TRAIN_DIR = 'train'
VALID_DIR = 'valid'
TEST_DIR = 'test'
IMAGE_SIZE = [416,416]

In [96]:
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

In [97]:
class CCDataset(Dataset):
  def __init__(self, mode = 'train', augmentation=None):
    if mode == 'train':
      self.dataset_path = TRAIN_DIR
      ann_path = os.path.join(TRAIN_DIR, '_annotations.coco.json')
    if mode == 'valid':
      self.dataset_path = VALID_DIR
      ann_path = os.path.join(VALID_DIR, '_annotations.coco.json')
    if mode == 'test':
      self.dataset_path = TEST_DIR
      ann_path = os.path.join(TEST_DIR, '_annotations.coco.json')
    
    self.coco = COCO(ann_path)
    self.cat_ids = self.coco.getCatIds()
    self.augmentation=augmentation

  def __len__(self):
      return len(self.coco.imgs)
  
  # memamnggil segmentasi 
  def get_masks(self, index):
      ann_ids = self.coco.getAnnIds([index])
      anns = self.coco.loadAnns(ann_ids)
      masks=[]

      for ann in anns:
            mask = self.coco.annToMask(ann)
            masks.append(mask)

      return masks
  
  # memamnggil bounding box
  def get_boxes(self, masks):
      num_objs = len(masks)
      boxes = []

      for i in range(num_objs):
          x,y,w,h = cv2.boundingRect(masks[i])
          boxes.append([x, y, x+w, y+h])

      return np.array(boxes)

# memamnggil gambar
  def __getitem__(self, index):
      # Load image
      img_info = self.coco.loadImgs([index])[0]
      image = cv2.imread(os.path.join(self.dataset_path,
                                    img_info['file_name']))
      # memanggil segmentasi sesuai index
      masks = self.get_masks(index)

      if self.augmentation:
         augmented = self.augmentation(image=image, masks=masks)
         image, masks = augmented['image'], augmented['masks']

      image = image.transpose(2,0,1) / 255.



      # Memasukkan segmentasi dan bounding box ke dalam file masks dan boxes
      masks = np.array(masks)
      boxes = self.get_boxes(masks)

      # Create target dict
      num_objs = len(masks)
      # membuat ektraksi ciri
      boxes = torch.as_tensor(boxes, dtype=torch.float32)
      labels = torch.ones((num_objs,), dtype=torch.int64)
      masks = torch.as_tensor(masks, dtype=torch.uint8)
      image = torch.as_tensor(image, dtype=torch.float32)
      data = {}
      data["boxes"] =  boxes
      data["labels"] = labels
      data["masks"] = masks

      return image, data

In [98]:
def collate_fn(batch):
  images = list()
  targets = list()
  for b in batch:
        images.append(b[0])
        targets.append(b[1])
  images = torch.stack(images, dim=0)
  return images, targets

In [99]:
import albumentations as A
import numpy as np # linear algebra
import pandas as pd

In [100]:
transform = A.Compose([
    # A.Resize(600, 600),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(
        contrast_limit=0.2, brightness_limit=0.3, p=0.5),
    A.OneOf([
        A.ImageCompression(p=0.8),
        A.RandomGamma(p=0.8),
        A.Blur(p=0.8),
        A.Equalize(mode='cv',p=0.8)
    ], p=1.0),
    A.OneOf([
        A.ImageCompression(p=0.8),
        A.RandomGamma(p=0.8),
        A.Blur(p=0.8),
        A.Equalize(mode='cv',p=0.8),
    ], p=1.0)
])

In [101]:
import torch
from tqdm import tqdm
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import datasets, models

In [102]:
# n_classes = 3
coco = COCO(os.path.join( "train", "_annotations.coco.json"))
categories = coco.cats
n_classes = len(categories.keys())
categories

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


{0: {'id': 0, 'name': 'Pest', 'supercategory': 'none'},
 1: {'id': 1, 'name': 'Belalang', 'supercategory': 'Pest'},
 2: {'id': 2, 'name': 'Sehat', 'supercategory': 'Pest'},
 3: {'id': 3, 'name': 'Ulat', 'supercategory': 'Pest'},
 4: {'id': 4, 'name': 'Wereng', 'supercategory': 'Pest'}}

In [103]:
def get_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(
            in_features, n_classes)
    model.to(DEVICE)
    
    return model

In [104]:
def save_checkpoint(state, filename="mask_rcnn.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [105]:
def load_checkpoint(checkpoint, model, optimizer, lr):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    #optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [106]:
if LOAD_MODEL and CHECKPOINT_FILE in os.listdir():
        print("Loading checkpoint")
        load_checkpoint(torch.load(CHECKPOINT_FILE), model, optimizer, LEARNING_RATE)

In [107]:
y = os.path.join( 'train')
y

'train'

In [108]:
train_dataset = CCDataset(mode='train', augmentation=transform)
train_loader = DataLoader(dataset=train_dataset,
                              batch_size=4,
                              shuffle=True,
                              # drop_last=True,
                              num_workers=4,
                              pin_memory=PIN_MEMORY,
                              collate_fn=collate_fn)

valid_dataset = CCDataset(mode='valid')
valid_loader = DataLoader(dataset=valid_dataset,
                              batch_size=2,
                              shuffle=False,
                              # drop_last=True,
                              pin_memory=PIN_MEMORY,
                              collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [109]:
from torchvision.utils import draw_bounding_boxes

In [110]:
def train_one_epoch(loader, model, optimizer, device):
    loop = tqdm(loader)

    all_losses = []
    all_losses_dict = []
    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )
    for batch_idx, (images, targets) in  enumerate(loop):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()
          
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)
          
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping trainig") # train if loss becomes infinity
            print(loss_dict)
            sys.exit(1)


        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    
    return(np.mean(all_losses))
    all_losses_dict = pd.DataFrame(all_losses_dict) # for printing
    # print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
    #     epoch, optimizer.param_groups[0]['lr'], np.mean(all_losses),
    #     all_losses_dict['loss_classifier'].mean(),
    #     all_losses_dict['loss_box_reg'].mean(),
    #     all_losses_dict['loss_rpn_box_reg'].mean(),
    #     all_losses_dict['loss_objectness'].mean()
    # ))


In [111]:
# def _get_iou_types(model):
#     model_without_ddp = model
#     if isinstance(model, torch.nn.parallel.DistributedDataParallel):
#         model_without_ddp = model.module
#     iou_types = ["bbox"]
#     if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
#         iou_types.append("segm")
#     if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
#         iou_types.append("keypoints")
#     return iou_types

In [112]:
# def evaluate(model, loader, device):
#     n_threads = torch.get_num_threads()
#     # FIXME remove this and make paste_masks_in_image run on the GPU
#     torch.set_num_threads(1)
#     # cpu_device = torch.device("cpu")
#     model.eval()
#     loop = tqdm(loader)
#     header = "Test:"

#     coco = get_coco_api_from_dataset(loop.dataset)
#     iou_types = _get_iou_types(model)
#     coco_evaluator = CocoEvaluator(coco, iou_types)

#     for batch_idx, (images, targets) in  enumerate(loop):
#         images = list(img.to(device) for img in images)

#         if torch.cuda.is_available():
#             torch.cuda.synchronize()
#         model_time = time.time()
#         outputs = model(images)

#         outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
#         model_time = time.time() - model_time

#         res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
#         evaluator_time = time.time()
#         coco_evaluator.update(res)
#         evaluator_time = time.time() - evaluator_time
#         metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

#     # gather the stats from all processes
#     metric_logger.synchronize_between_processes()
#     print("Averaged stats:", metric_logger)
#     coco_evaluator.synchronize_between_processes()

#     # accumulate predictions from all images
#     coco_evaluator.accumulate()
#     coco_evaluator.summarize()
#     torch.set_num_threads(n_threads)
#     return coco_evaluator

In [113]:
best_vloss = np.inf
def validate(loader, model, optimizer, device, epoch):
    global best_vloss
    loop = tqdm(loader)
    running_vloss = 0
    for batch_idx, (images, targets) in enumerate(loop):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        with torch.no_grad():
          loss_dict = model(images, targets)
        
        losses = sum(loss for loss in loss_dict.values())
        running_vloss += losses
        
    avg_vloss = running_vloss / (batch_idx + 1)
    print(f"Avg Valid Loss: {avg_vloss}")
    if avg_vloss < best_vloss:
      best_vloss = avg_vloss
      # if SAVE_MODEL:
      #       print("Model improved, saving...")
      #       checkpoint = {
      #           "state_dict": model.state_dict(),
      #           "optimizer": optimizer.state_dict(),
      #       }
      #       save_checkpoint(checkpoint, filename=f"1152KaggleBest_second_{epoch}.pth.tar")
    print('\n')
    return avg_vloss

In [114]:
model = get_model()
optimizer = torch.optim.AdamW(params=model.parameters(),
                                  lr=LEARNING_RATE,
                                  weight_decay=WEIGHT_DECAY)

# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

In [ ]:
model.train()
train_error =[]
# val_error = []
num_epochs = 25
for epoch in range(num_epochs):
                print(f"Epoch: {epoch}")
                train = train_one_epoch(train_loader, model, optimizer, DEVICE)
                # vloss= validate(valid_loader, model, optimizer, DEVICE, epoch)
                train_error.append(train*100)
                # val_error.append(vloss.tolist()*100)

Epoch: 0


 71%|███████   | 29/41 [00:26<00:10,  1.11it/s]

In [ ]:
plt.subplot(2, 1, 2)
plt.plot(train_error, label='Training Loss')
# plt.plot(val_error, label='Validation Loss')
plt.legend()
plt.title('Training Loss')
plt.axvline(num_epochs, color="gray", label="Epoch")
# plt.ylabel('merror')
plt.ylabel('Loss')

plt.show()

In [ ]:
# train

In [ ]:
# print("Score")
# for i in range(2):
#   img, _ = valid_dataset[i]
#   img_int = torch.tensor(img*255, dtype=torch.uint8)
#   with torch.no_grad():
#       prediction = model([img.to(DEVICE)])
#       pred = prediction[0]
  
#   print( [classes[i] for i in pred['labels'][pred['scores'] > 0.8].tolist()], ": ",pred['scores'].tolist() )
#   fig = plt.figure(figsize=(14, 10))
#   plt.imshow(draw_bounding_boxes(img_int,
#     pred['boxes'][pred['scores'] > 0.8], 
#     [classes[i] for i in pred['labels'][pred['scores'] > 0.8].tolist()], width=4
#   ).permute(1, 2, 0))

  
  
    
  


In [ ]:
# vloss

In [ ]:
import cv2

In [ ]:
classes = [i[1]['name'] for i in categories.items()]
classes

In [ ]:
def predict_single_frame(frame):
    images = cv2.resize(frame, IMAGE_SIZE, cv2.INTER_LINEAR)/255
    images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
    images = images.swapaxes(1, 3).swapaxes(2, 3)
    images = list(image.to(DEVICE) for image in images)
    
    with torch.no_grad():
      pred = model(images)
    
    print(pred)
    
    im = images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.float32)
    im2 = np.zeros_like(im).astype(np.float32)
    for i in range(len(pred[0]['masks'])):
        msk=pred[0]['masks'][i,0].detach().cpu().numpy()
        scr=pred[0]['scores'][i].detach().cpu().numpy()
        box=pred[0]['boxes'][i].detach().cpu().numpy()
        lbl=pred[0]['labels'][i].detach().cpu().numpy()
        if (lbl == 1) or (lbl == 3) or (lbl == 4) :
          x = "hama"
        else :
          x = "tidak hama"
        print(lbl)
        if scr>0.9 :
            cv2.rectangle(im, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0,0,1), 2)
            cv2.putText(im, "{0:.2f}%".format(scr*100), (int(box[0]+10), int(box[1])+40), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,0,1), 2, cv2.LINE_AA)
            cv2.putText(im, x , (int(box[0]+4), int(box[1])+15), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,0,1), 2, cv2.LINE_AA)
            im2[:,:,0][msk>0.87] = np.random.uniform(0,1)
            im2[:, :, 1][msk > 0.87] = np.random.uniform(0,1)
            im2[:, :, 2][msk > 0.87] = np.random.uniform(0,1)

    return (cv2.addWeighted(im, 0.8, im2, 0.2,0)*255).astype(np.uint8)

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Ta/Mask_RCNN/Dataset/Uji/2023-01-22_12.17.14.272.png')
model.train(False)

if (cap.isOpened()== False): 
    print("Error opening video stream or file")

images = []   
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        result_frame = predict_single_frame(frame)
        images.append(result_frame)
    else: 
        break

cap.release()

In [ ]:
# result_frame

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(result_frame)

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Ta/Mask_RCNN/Dataset/Uji/2023-01-22_11.55.18.642.png')
model.train(False)

if (cap.isOpened()== False): 
    print("Error opening video stream or file")

images = []   
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        result_frame = predict_single_frame(frame)
        images.append(result_frame)
    else: 
        break

cap.release()

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(result_frame)

In [ ]:
t = torch.tensor([1,2,3]) # cara 1
print(t)